In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import applications, layers, Input
from tensorflow.keras.models import Sequential
import keras.backend.tensorflow_backend as tfback
import pandas as pd

root_path = '/content/drive/My Drive/DataSet/MRNET data set/MRNet-v1.0'



In [0]:
def save_example(root_path, task, view, model_name, num, features):
  index = '0' * (4 - len(str(num))) + str(num)
  print('saving example:', index)
  np.save(open(f"{root_path}/{task}/{view}/{model_name}/{index}.npy", 'wb'), features)

def load_labels(root_path, task, result_label):
  records = pd.read_csv(root_path + f"/{task}-{result_label}.csv", header=None, names=['id', 'label'])
  return records['label'].to_numpy()

def load_features(root_path, task, view, model_name, start, end, dim):
  features = np.empty((0, dim))
  for i in range(start, end):
    index = '0' * (4 - len(str(i))) + str(i)
    path = root_path + f"/{task}/{view}/{model_name}/{index}.npy"
    print('loading', index, 'features')
    x = np.load(path).reshape((1, dim))
    features = np.concatenate((features, x), axis = 0)
  return features

def plot_logs(logs, save_path):
  fig = plt.figure(figsize=(20, 10))
  plt.ylim(0, 1)
  plt.plot(logs['loss'], 'g', label="train losses")
  plt.plot(logs['val_loss'], 'r', label="val losses")
  plt.grid(True)
  plt.title('Training loss vs. Validation loss')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()
  plt.savefig(save_path, bbox_inches='tight')


Custom layers:

In [0]:
class Max(keras.layers.Layer):
  def __init__(self, **kwargs):
    super(Max, self).__init__(**kwargs)

  def call(self, inputs):
    self.outputs = tf.math.reduce_max(
        inputs, axis=0, keepdims=True
    )
    return self.outputs


class Squeeze(keras.layers.Layer):
  def __init__(self,**kwargs):
    super(Squeeze, self).__init__(**kwargs)

  def call(self, inputs):
    self.outputs = tf.squeeze(inputs, axis = 0)
    return self.outputs


VGG16 from-scratch implementation:

In [0]:
# function for creating a vgg block
def vgg_block(n_filters, n_conv):
  # add convolutional layers 
  block = [layers.Conv2D(n_filters, (3,3), padding='same', activation='relu') for i in range(n_conv)]
	# add max pooling layer
  block.append(layers.MaxPooling2D((2,2), strides=(2,2)))
  return block

def vgg_layers():
  vgg_layers = vgg_block(64, 2)
  vgg_layers.extend(vgg_block(128, 2))
  vgg_layers.extend(vgg_block(256, 3))
  vgg_layers.extend(vgg_block(512, 3))
  vgg_layers.extend(vgg_block(512, 3))

  return vgg_layers


class VGG16():
  def __init__(self, root_path, view, result_label, task = 'train'):
    self.result_label = result_label
    self.root_path = root_path
    self.view = view
    self.task = task
    #Define The ConvNet: self.vgg_layers
    self.vgg_layers = vgg_layers()

    inputs = keras.Input(shape=(None, 3, 256, 256))
    x = Squeeze()(inputs)
    x = layers.experimental.preprocessing.Rescaling(scale=1.0 / 255)(x)
    x = layers.experimental.preprocessing.RandomFlip(mode='horizontal')
    for layer in self.vgg_layers:
      x = layer(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = Max()(x)

    outputs = layers.Dense(
        1, activation='sigmoid', 
    )(x)

    self.vgg_model = keras.Model(inputs, outputs)

    self.vgg_model.compile(
        optimizer=keras.optimizers.Adam(1e-2), loss='binary_crossentropy', metrics=[keras.metrics.AUC()]
        )
    

  def __save_history(self):
    with open(self.root_path + f'/models/vgg16-scratch-{self.view}-{self.result_label}-history', 'wb') as file_pi:
      pickle.dump(self.vgg_model_logs, file_pi)
    
  def __load_labels(self):
    return load_labels(self.root_path, self.task, self.result_label)

  def train_model(self, X = None, save_model = False):
    self.task = 'train'
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
        min_delta=0,
        patience=15,
        verbose=0, mode='auto',
        restore_best_weights=True)
    
    reduce_lr = keras.callbacks.ReduceLROnPlateau(
      monitor='val_loss', factor=0.1, patience=5, verbose=0, mode='auto',
      min_delta=0, cooldown=0, min_lr=1e-5
    )
    params = {'shape': (256, 256),
        'n_channels': 3,
        'batch_size': 1,
        'shuffle': True,
        'extractor': None,
        'task': self.task, 
        'view': self.view, 
        'result_label': self.result_label
    }

    train_generator = MRNetDataGenerator(**params, bounds=(0, 1017), X = X)
    valid_generator = MRNetDataGenerator(**params, bounds=(1017, 1130), X = X)
    history = self.vgg_model.fit(
        train_generator, batch_size=1, epochs=35, verbose=2, callbacks=[reduce_lr, early_stopping],
        validation_data=valid_generator
    )
    self.vgg_model_logs = history.history
    if save_model == True:
      self.vgg_model.save(self.root_path + f'/models/vgg16-scratch-{self.view}-{self.result_label}.h5')
      self.__save_history()

    return self.vgg_model_logs






ResNet from-scratch implementation:

In [0]:
def identity_block(X, f, filters, stage, block):
  # defining name basis
  conv_name_base = 'res' + str(stage) + block + '_branch'
  bn_name_base = 'bn' + str(stage) + block + '_branch'

  # Retrieve Filters
  F1, F2, F3 = filters

  # Save the input value. We'll need this later to add back to the main path. 
  X_shortcut = X

  # First component of main path
  X = layers.Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
  X = layers.BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
  X = layers.Activation('relu')(X)

  # Second component of main path
  X = layers.Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
  X = layers.BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
  X = layers.Activation('relu')(X)

  # Third component of main path
  X = layers.Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
  X = layers.BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

  # Final step: Add shortcut value to main path, and pass it through a RELU activation
  X = layers.Add()([X, X_shortcut])
  X = layers.Activation('relu')(X)

  return X

def convolutional_block(X, f, filters, stage, block, s = 2):
  # defining name basis
  conv_name_base = 'res' + str(stage) + block + '_branch'
  bn_name_base = 'bn' + str(stage) + block + '_branch'
    
  # Retrieve Filters
  F1, F2, F3 = filters
    
  # Save the input value
  X_shortcut = X


  ##### MAIN PATH #####
  # First component of main path 
  X = layers.Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
  X = layers.BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
  X = layers.Activation('relu')(X)

  # Second component of main path
  X = layers.Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
  X = layers.BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
  X = layers.Activation('relu')(X)

  # Third component of main path
  X = layers.Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
  X = layers.BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    
  ##### SHORTCUT PATH ####
  X_shortcut = layers.Conv2D(F3, (1, 1), strides = (s,s), name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
  X_shortcut = layers.BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

  # Final step: Add shortcut value to main path, and pass it through a RELU activation
  X = layers.Add()([X, X_shortcut])
  X = layers.Activation('relu')(X)
    
  return X

def ResNet50(X_input, classes = 1):   
    # Define the input as a tensor with shape input_shape

    # Zero-Padding
    X = layers.ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = layers.Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = layers.BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = layers.Activation('relu')(X)
    X = layers.MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    return X


class ResNet():
  def __init__(self, root_path, view, result_label, task = 'train'):
    self.result_label = result_label
    self.root_path = root_path
    self.view = view
    self.task = task

    inputs = keras.Input(shape=(None, 3, 256, 256))
    x = Squeeze()(inputs)
    x = layers.experimental.preprocessing.Rescaling(scale=1.0 / 255)(x)
    x = layers.experimental.preprocessing.RandomFlip(mode='horizontal')
    x = ResNet50(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = Max()(x)

    outputs = layers.Dense(
        1, activation='sigmoid', 
    )(x)

    self.resnet_model = keras.Model(inputs, outputs)

    self.resnet_model.compile(
        optimizer=keras.optimizers.Adam(1e-2), loss='binary_crossentropy', metrics=[keras.metrics.AUC()]
        )
    
    print(self.resnet_model.summary())

  def __save_history(self):
    with open(self.root_path + f'/models/resnet-scratch-{self.view}-{self.result_label}-history', 'wb') as file_pi:
      pickle.dump(self.resnet_model_logs, file_pi)
    
  def __load_labels(self):
    return load_labels(self.root_path, self.task, self.result_label)

  def train_model(self, X = None, save_model = False):
    self.task = 'train'
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
        min_delta=0,
        patience=15,
        verbose=0, mode='auto',
        restore_best_weights=True)
    
    reduce_lr = keras.callbacks.ReduceLROnPlateau(
      monitor='val_loss', factor=0.1, patience=5, verbose=0, mode='auto',
      min_delta=0, cooldown=0, min_lr=1e-5
    )
    params = {'shape': (256, 256),
        'n_channels': 3,
        'batch_size': 1,
        'shuffle': True,
        'extractor': None,
        'task': self.task, 
        'view': self.view, 
        'result_label': self.result_label
    }

    train_generator = MRNetDataGenerator(**params, bounds=(0, 1017), X = X)
    valid_generator = MRNetDataGenerator(**params, bounds=(1017, 1130), X = X)
    history = self.resnet_model.fit(
        train_generator, batch_size=1, epochs=20, verbose=2, callbacks=[reduce_lr, early_stopping],
        validation_data=valid_generator
    )
    self.resnet_model_logs = history.history
    if save_model == True:
      self.resnet_model.save(self.root_path + f'/models/resnet-scratch-{self.view}-{self.result_label}.h5')
      self.__save_history()

    return self.resnet_model_logs






Inception V3 from-scratch implementation:

In [0]:
def conv2d_bn(x, filters, num_row, num_col, padding='same', strides=(1, 1), name=None):
    """Utility function to apply conv + BatchNormalization.
       BatchNormalization : Normalize the activations of the previous layer at each batch,
        i.e. applies a transformation that maintains
         the mean activation close to 0 and the activation standard deviation close to 1.
    Arguments:
        x: input tensor.
        filters: filters in `Conv2D`.
        num_row: height of the convolution kernel.
        num_col: width of the convolution kernel.
        padding: padding mode in `Conv2D`.
        strides: strides in `Conv2D`.
        name: name of the ops; will become `name + '_conv'`
            for the convolution and `name + '_bn'` for the
            batch norm layer.
    Returns:
        Output tensor after applying `Conv2D` and `BatchNormalization`.
    """
    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
    '''
    if K.image_data_format() == 'channels_first':
        bn_axis = 1
    else:
        bn_axis = 3
    '''
    bn_axis = 1
    x = layers.Conv2D(
        filters, (num_row, num_col),
        strides=strides,
        padding=padding,
        use_bias=False,
        name=conv_name)(x)
    x = layers.BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    x = layers.Activation('relu', name=name)(x)
    return x

def InceptionV3_from_scratch(input_layer):

  #channel depth dimension
  channel_axis = 1
      
  x = conv2d_bn(input_layer, 32, 3, 3, strides=(2, 2), padding='valid')
  x = conv2d_bn(x, 32, 3, 3, padding='valid')
  x = conv2d_bn(x, 64, 3, 3)
  x = MaxPooling2D((3, 3), strides=(2, 2))(x)

  x = conv2d_bn(x, 80, 1, 1, padding='valid')
  x = conv2d_bn(x, 192, 3, 3, padding='valid')
  x = MaxPooling2D((3, 3), strides=(2, 2))(x)

  # mixed 0, 1, 2: 35 x 35 x 256
  branch1x1 = conv2d_bn(x, 64, 1, 1)

  branch5x5 = conv2d_bn(x, 48, 1, 1)
  branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

  branch3x3dbl = conv2d_bn(x, 64, 1, 1)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

  branch_pool = layers.AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
  branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
  x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed0')
  
  # mixed 1: 35 x 35 x 256
  branch1x1 = conv2d_bn(x, 64, 1, 1)

  branch5x5 = conv2d_bn(x, 48, 1, 1)
  branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

  branch3x3dbl = conv2d_bn(x, 64, 1, 1)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

  branch_pool = layers.AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
  branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
  x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed1')
  # mixed 2: 35 x 35 x 256
  branch1x1 = conv2d_bn(x, 64, 1, 1)

  branch5x5 = conv2d_bn(x, 48, 1, 1)
  branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

  branch3x3dbl = conv2d_bn(x, 64, 1, 1)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

  branch_pool = layers.AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
  branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
  x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed2')

  # mixed 3: 17 x 17 x 768
  branch3x3 = conv2d_bn(x, 384, 3, 3, strides=(2, 2), padding='valid')

  branch3x3dbl = conv2d_bn(x, 64, 1, 1)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
  branch3x3dbl = conv2d_bn(
        branch3x3dbl, 96, 3, 3, strides=(2, 2), padding='valid')

  branch_pool = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)
  x = layers.concatenate(
        [branch3x3, branch3x3dbl, branch_pool], axis=channel_axis, name='mixed3')

  # mixed 4: 17 x 17 x 768
  branch1x1 = conv2d_bn(x, 192, 1, 1)

  branch7x7 = conv2d_bn(x, 128, 1, 1)
  branch7x7 = conv2d_bn(branch7x7, 128, 1, 7)
  branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

  branch7x7dbl = conv2d_bn(x, 128, 1, 1)
  branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
  branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 1, 7)
  branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
  branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

  branch_pool = layers.AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
  branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
  x = layers.concatenate(
        [branch1x1, branch7x7, branch7x7dbl, branch_pool],
        axis=channel_axis,
        name='mixed4')

  # mixed 5, 6: 17 x 17 x 768
  for i in range(2):
      branch1x1 = conv2d_bn(x, 192, 1, 1)

      branch7x7 = conv2d_bn(x, 160, 1, 1)
      branch7x7 = conv2d_bn(branch7x7, 160, 1, 7)
      branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

      branch7x7dbl = conv2d_bn(x, 160, 1, 1)
      branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 7, 1)
      branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 1, 7)
      branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 7, 1)
      branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

      branch_pool = layers.AveragePooling2D(
            (3, 3), strides=(1, 1), padding='same')(x)
      branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
      x = layers.concatenate(
            [branch1x1, branch7x7, branch7x7dbl, branch_pool],
            axis=channel_axis,
            name='mixed' + str(5 + i))

  # mixed 7: 17 x 17 x 768
  branch1x1 = conv2d_bn(x, 192, 1, 1)

  branch7x7 = conv2d_bn(x, 192, 1, 1)
  branch7x7 = conv2d_bn(branch7x7, 192, 1, 7)
  branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

  branch7x7dbl = conv2d_bn(x, 192, 1, 1)
  branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 7, 1)
  branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)
  branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 7, 1)
  branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

  branch_pool = layers.AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
  branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
  x = layers.concatenate(
        [branch1x1, branch7x7, branch7x7dbl, branch_pool],
        axis=channel_axis,
        name='mixed7')

  # mixed 8: 8 x 8 x 1280
  branch3x3 = conv2d_bn(x, 192, 1, 1)
  branch3x3 = conv2d_bn(branch3x3, 320, 3, 3,
                          strides=(2, 2), padding='valid')

  branch7x7x3 = conv2d_bn(x, 192, 1, 1)
  branch7x7x3 = conv2d_bn(branch7x7x3, 192, 1, 7)
  branch7x7x3 = conv2d_bn(branch7x7x3, 192, 7, 1)
  branch7x7x3 = conv2d_bn(
        branch7x7x3, 192, 3, 3, strides=(2, 2), padding='valid')

  branch_pool = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)
  x = layers.concatenate(
        [branch3x3, branch7x7x3, branch_pool], axis=channel_axis, name='mixed8')

  # mixed 9: 8 x 8 x 2048
  for i in range(2):
        branch1x1 = conv2d_bn(x, 320, 1, 1)

        branch3x3 = conv2d_bn(x, 384, 1, 1)
        branch3x3_1 = conv2d_bn(branch3x3, 384, 1, 3)
        branch3x3_2 = conv2d_bn(branch3x3, 384, 3, 1)
        branch3x3 = layers.concatenate(
            [branch3x3_1, branch3x3_2], axis=channel_axis, name='mixed9_' + str(i))

        branch3x3dbl = conv2d_bn(x, 448, 1, 1)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 384, 3, 3)
        branch3x3dbl_1 = conv2d_bn(branch3x3dbl, 384, 1, 3)
        branch3x3dbl_2 = conv2d_bn(branch3x3dbl, 384, 3, 1)
        branch3x3dbl = layers.concatenate(
            [branch3x3dbl_1, branch3x3dbl_2], axis=channel_axis)

        branch_pool = layers.AveragePooling2D(
            (3, 3), strides=(1, 1), padding='same')(x)
        branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
        x = layers.concatenate(
            [branch1x1, branch3x3, branch3x3dbl, branch_pool],
            axis=channel_axis,
            name='mixed' + str(9 + i))
  

  return x


class InceptionV3():
  def __init__(self, root_path, view, result_label, task = 'train'):
    self.result_label = result_label
    self.root_path = root_path
    self.view = view
    self.task = task

    inputs = keras.Input(shape=(None, 3, 256, 256))
    x = Squeeze()(inputs)
    x = layers.experimental.preprocessing.Rescaling(scale=1.0 / 255)(x)
    x = layers.experimental.preprocessing.RandomFlip(mode='horizontal')
    x = InceptionV3_from_scratch(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = Max()(x)

    outputs = layers.Dense(
        1, activation='sigmoid', 
    )(x)

    self.inceptionv3_model = keras.Model(inputs, outputs)

    self.inceptionv3_model.compile(
        optimizer=keras.optimizers.Adam(1e-2), loss='binary_crossentropy', metrics=[keras.metrics.AUC()]
        )
    
    print(self.inceptionv3_model.summary())

  def __save_history(self):
    with open(self.root_path + f'/models/inceptionv3-scratch-{self.view}-{self.result_label}-history', 'wb') as file_pi:
      pickle.dump(self.inceptionv3_model_logs, file_pi)
    
  def __load_labels(self):
    return load_labels(self.root_path, self.task, self.result_label)

  def train_model(self, X = None, save_model = False):
    self.task = 'train'
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
        min_delta=0,
        patience=15,
        verbose=0, mode='auto',
        restore_best_weights=True)
    
    reduce_lr = keras.callbacks.ReduceLROnPlateau(
      monitor='val_loss', factor=0.1, patience=5, verbose=0, mode='auto',
      min_delta=0, cooldown=0, min_lr=1e-5
    )
    params = {'shape': (256, 256),
        'n_channels': 3,
        'batch_size': 1,
        'shuffle': True,
        'extractor': None,
        'task': self.task, 
        'view': self.view, 
        'result_label': self.result_label
    }

    train_generator = MRNetDataGenerator(**params, bounds=(0, 1017), X = X)
    valid_generator = MRNetDataGenerator(**params, bounds=(1017, 1130), X = X)
    history = self.inceptionv3_model.fit(
        train_generator, batch_size=1, epochs=20, verbose=2, callbacks=[reduce_lr, early_stopping],
        validation_data=valid_generator
    )
    self.inceptionv3_model_logs = history.history
    if save_model == True:
      self.inceptionv3_model.save(self.root_path + f'/models/inceptionv3-scratch-{self.view}-{self.result_label}.h5')
      self.__save_history()

    return self.inceptionv3_model_logs

TransferLearning Class:

In [0]:
"""Transfer Learning"""
class TransferLearningModel():
  def __init__(self, root_path, view, result_label, model_name, task = 'train'):
    self.result_label = result_label
    self.root_path = root_path
    self.view = view
    self.task = task
    self.model_name = model_name
    #Define The ConvNet: self.cnn_layer
    if self.model_name == 'vgg16':
      self.cnn_layer = applications.VGG16(
          include_top=False,
          weights="imagenet",
          input_shape=(3, 256,256),
          pooling='avg'
      )
      self.output_dim = 512
    elif self.model_name == 'resnet':
      self.cnn_layer = keras.applications.ResNet50(
        include_top=False,
        weights="imagenet",
        input_shape=(3,256,256),
        pooling='avg',
      )
      self.output_dim = 2048
    elif self.model_name == 'inceptionv3':
      self.cnn_layer = keras.applications.InceptionV3(
        include_top=False,
        weights="imagenet",
        input_shape=(3,256,256),
        pooling='avg',
      )
      self.output_dim = 2048
    self.cnn_layer.trainable = False

    #Define the input layer + the convnet + the maxpool layer 
    #   : self.extractor
    inputs = keras.Input(shape=(None, 3, 256, 256))
    x = Squeeze()(inputs)
    if model_name == 'vgg16':
      x = applications.vgg16.preprocess_input(x)
    elif model_name == 'resnet':
      x = applications.resnet.preprocess_input(x)
    elif model_name == 'inceptionv3':
      x = applications.inception_v3.preprocess_input(x)

    x = self.cnn_layer(x, training=False)
    outputs = Max()(x)
    self.extractor = keras.Model(inputs, outputs)
    self.extractor.compile()

    #Define the FC layer: self.fc
    inputs = keras.Input(shape=(self.output_dim,))
    outputs = layers.Dense(
        1, activation='sigmoid', 
        kernel_regularizer=keras.regularizers.l2(0.1), 
        bias_regularizer=keras.regularizers.l2(0.1)
    )(inputs)
    self.fc = keras.Model(inputs, outputs)

  def __save_example(self, num, features):
    save_example(self.root_path, self.task, self.view,self.model_name , num, features)

  def __load_labels(self):
    return load_labels(self.root_path, self.task, self.result_label)



  def __load_features(self):
    if self.task == 'train':
      start = 0
      end = 1130
    else:
      start = 1130
      end = 1250
    
    self.features = load_features(self.root_path, self.task, self.view,
                                   self.model_name, start, end, self.output_dim)
    return self.features
  
  def __save_history(self):
    with open(self.root_path + f'/models/{self.model_name}-auc-fc-{self.view}-{self.result_label}-history', 'wb') as file_pi:
      pickle.dump(self.train_fc_logs, file_pi)

  def plot_logs(self):
    fig = plt.figure(figsize=(20, 10))
    plt.ylim(0, 1)
    plt.plot(self.train_fc_logs['loss'], 'g', label="train losses")
    plt.plot(self.train_fc_logs['val_loss'], 'r', label="val losses")
    plt.grid(True)
    plt.title('Training loss vs. Validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(self.root_path + f'/figures/{self.model_name}-{self.view}-{self.result_label}.png', bbox_inches='tight')


  def __load_history(self):
    return pickle.load(open(self.root_path + f'/models/{self.model_name}-auc-fc-{self.view}-{self.result_label}-history', 'rb'))

  def load_fc(self):
    self.fc = keras.models.load_model(
      self.root_path + f'/models/{self.model_name}-auc-fc-{self.view}-{self.result_label}.h5', compile=True
      )
    self.train_fc_logs = self.__load_history()

  def extract_features(self, task, save = False, X = None):
    self.task = task
    self.features = np.empty((0,512))
    if X != None:
      for i in range(1130):
        print(f'extracting {i} (Memory Mode)')
        x = X[i]
        x = tf.expand_dims(x, 0)
        Y = self.extractor.predict(
          x, batch_size=1, verbose=0
        )
        self.features = np.concatenate((self.features,Y) , axis = 0)
    else:
      params = {'shape': (256, 256),
          'n_channels': 3,
          'batch_size': 1,
          'shuffle': False,
          'extractor': None,
          'task': self.task, 
          'view': self.view, 
          'result_label': self.result_label}

      predict_generator = MRNetDataGenerator(**params)
      self.features = self.extractor.predict(
        predict_generator, verbose=0
      )

    if save == True:
      shift = 0
      if self.task == 'valid':
        shift = 1130
      for example_num, example_features in enumerate(self.features):
        self.__save_example(example_num + shift, example_features)

    return self.features

  def predict_fc(self):
    self.load_fc()
    X = self.__load_features()
    self.predictions = self.fc.predict(
      X, verbose=1
    )
    return self.predictions


  def train_fc(self, memory = False, save_fc = False):
    self.task = 'train'
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
        min_delta=0,
        patience=5,
        verbose=0, mode='auto',
        restore_best_weights=True)
    
    reduce_lr = keras.callbacks.ReduceLROnPlateau(
      monitor='val_loss', factor=0.1, patience=3, verbose=0, mode='auto',
      min_delta=0, cooldown=0, min_lr=1e-5
    )
    
    if memory == True:
      Y = self.__load_labels()
      X = self.__load_features()
      self.fc.compile(
        optimizer=keras.optimizers.Adam(1e-4), loss='binary_crossentropy', metrics=[keras.metrics.AUC()]
      )
      history = self.fc.fit(
          x=X, y=Y, batch_size=1, epochs=35, verbose=2, callbacks=[reduce_lr, early_stopping],
          validation_split=0.1, validation_data=None, shuffle=True,
      )
      self.train_fc_logs = history.history

    else:
      pass
    if save_fc == True:
      self.fc.save(self.root_path + f'/models/{self.model_name}-auc-fc-{self.view}-{self.result_label}.h5')
      self.__save_history()

    return self.train_fc_logs




    
    reduce_lr = keras.callbacks.ReduceLROnPlateau(
      monitor='val_loss', factor=0.5, patience=5, verbose=0, mode='auto',
      min_delta=0, cooldown=0, min_lr=1e-7
    )


      